In [2]:
%load_ext autoreload
%autoreload 2

In [35]:

import os
os.chdir("/Users/bytedance/MLTrader-deploy")
from research.orderbook_strategies.utils import helper
from research.orderbook_strategies.utils import factor_analysis
from research.orderbook_strategies.utils.product_info import *
import numpy as np
import pandas as pd
import itertools

In [4]:
product = "BTCUSDT"
signal_name = "dbook.atr.64"
file_name = "2023-06-01"
all_signal = helper.auto_get_alldates_signal(signal_name, product)
tranct = product_info[product]["tranct"]  ## transaction cost of the product
tranct_ratio = product_info[product][
    "tranct.ratio"
]  ## True: based on notional; False: fixed tranct
open_list = np.quantile(
    abs(all_signal),
    np.append(np.linspace(0.8, 0.99, 5), np.linspace(0.991, 0.999, 5)),
)  ## open threshold
thre_list = []
for cartesian in itertools.product(
    open_list, np.array([0.2, 0.4, 0.6, 0.8, 1.0])
):  ## close threshold
    thre_list.append((cartesian[0], -cartesian[0] * cartesian[1]))
thre_list = np.array(thre_list)
thre_mat = pd.DataFrame(
    data=OrderedDict([("open", thre_list[:, 0]), ("close", thre_list[:, 1])])
)  ## threshold matrix
S = helper.get_good_signal(product, signal_name, date_str=file_name)
pred = np.asarray(S)
data = helper.get_data(
    product,
    file_name,
    columns=["time", "good", "bid", "ask", "next.bid", "next.ask", "atr.4096", "ret"],
)
atr = data["atr.4096"][data["good"]].reset_index(drop=True)
data = data[data["good"]].reset_index(drop=True)
results = []
atr_filter = 0.01
max_spread = 0.1

In [5]:
# data = data.iloc[4000: 5000]
# atr = atr.iloc[4000: 5000]
# S = S[4000: 5000]

In [6]:
thre = thre_mat.iloc[5]
print(thre)

open     0.000019
close   -0.000004
Name: 5, dtype: float64


In [7]:
result, position_pos1 = factor_analysis.backtest(pred,
            data,
            thre=(thre[0], thre[1]),
            atr=atr,
            atr_filter=atr_filter,
            tranct_ratio=tranct_ratio,
            max_spread=max_spread,
            tranct=tranct)

In [72]:
# result2 = factor_analysis.backtest_loop(pred,
#             data,
#             thre=(thre[0], thre[1]),
#             atr=atr,
#             atr_filter=atr_filter,
#             tranct_ratio=tranct_ratio,
#             max_spread=max_spread,
#             tranct=tranct)

In [36]:
pred = S
thre = (thre[0], thre[1])

cur_spread = data["ask"] - data["bid"]
buy = pred > thre[0]
sell = pred < -thre[0]
date_ = pd.to_datetime(data["time"].iloc[0]).date()
signal = pd.Series(data=0, index=data.index)
position = signal.copy()
signal[buy] = 1
signal[sell] = -1
if atr is not None and atr_filter is not None:
    signal[atr < atr_filter] = 0
scratch = -thre[1]
position_pos = pd.Series(data=np.nan, index=data.index)
position_pos.iloc[0] = 0
position_pos[
    (signal == 1)
    & (data["next.ask"] > 0)
    & (data["next.bid"] > 0)
    & (cur_spread < max_spread)
] = 1

position_pos[
    (pred < -scratch) 
    & (data["next.bid"] > 0) 
    & (cur_spread < max_spread)
] = 0

position_pos.ffill(inplace=True)

position_neg = pd.Series(data=np.nan, index=data.index)
position_neg.iloc[0] = 0
position_neg[
    (signal == -1)
    & (data["next.ask"] > 0)
    & (data["next.bid"] > 0)
    & (cur_spread < max_spread)
] = -1
position_neg[
    (pred > scratch) & (data["next.ask"] > 0) & (cur_spread < max_spread)
] = 0
position_neg.ffill(inplace=True)

In [37]:
print(position_pos.sum())
print(position_neg.sum())

222.0
-295.0


In [42]:
results, position_pos2 = factor_analysis.backtest_par(pred,
            data,
            thre=[(thre[0], thre[1]) for thre in thre_mat.values] ,
            atr=atr,
            atr_filter=atr_filter,
            tranct_ratio=tranct_ratio,
            max_spread=max_spread,
            tranct=tranct)

In [43]:
results[5]

DailyResult(date=datetime.date(2023, 6, 1), num=249, avg_pnl=-22.0503530923662, final_pnl=-5490.537919999184, avg_ret=-0.000819407671750609, ret=-0.20403251026590163)

In [44]:
result

DailyResult(date=datetime.date(2023, 6, 1), num=249, avg_pnl=-22.0503530923662, final_pnl=-5490.537919999184, avg_ret=-0.000819407671750609, ret=-0.20403251026590163)

In [13]:
print(position_pos2[:, 5].sum())

7984.0


In [12]:
print(position_pos1.sum())

8560.0


In [24]:
np.nonzero(position_pos1.values != position_pos2[:, 5])[0][:10]

array([4702, 4703, 4704, 4705, 4754, 4755, 4756, 4757, 4758, 4759])

In [29]:
position_pos1[4700: 4710]

4700    1.0
4701    1.0
4702    1.0
4703    1.0
4704    1.0
4705    1.0
4706    0.0
4707    0.0
4708    0.0
4709    0.0
dtype: float64

In [30]:
position_pos2[4700: 4710, 5]

array([1., 1., 0., 0., 0., 0., 0., 0., 0., 0.])

In [40]:
S[4700: 4710] < thre[1]

array([False, False, False, False, False, False,  True,  True,  True,
        True])

In [39]:
S[4700: 4710]

array([ 1.54429159e-05,  8.58921485e-06,  1.80933897e-06,  1.78984886e-06,
        1.77056880e-06,  1.75149615e-06, -4.95472685e-06, -1.15887127e-05,
       -4.77652107e-06, -4.72506766e-06])

In [41]:
thre[1]

-3.7300466912107265e-06

In [48]:
from pathlib import Path
res = helper.load(Path("/Volumes/AdamDrive/experiment_root/orderbook_research/signal_results/signal_result_with_atr/OPUSDT.kdj.j.std.512.reverse.pkl"))